In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [167]:
df = pd.read_csv("assets.csv")
df = df[df["type"] == "model"]
df.drop(columns=["sample", "datasheet", "included", "excluded", "adaptation", "output_space", "terms_of_service", "monthly_active_users", "user_distribution", "failures"], inplace=True)
df["created_date"] = pd.to_datetime(df["created_date"], format="%d/%m/%Y")
df["inputs"] = df.modality.str.split("; ").str[0].str.split(", ")
df["outputs"] = df.modality.str.split("; ").str[1].str.split(", ")
df.drop(columns="modality", inplace = True)
df["size"] = df["size"].apply(lambda x: np.nan if x == "unknown" else x)
df["dense"] = df["size"].str.endswith("(dense)")
df["dense"].fillna(False, inplace=True)
df["parameters"] = df["size"].str.extract(r"^(\d+[a-zA-Z])")
df["parameters_(millions)"] = df["parameters"].str.replace("M", "").str.replace("B", "000").str.replace("T", "000000")
df["parameters_(millions)"] = pd.to_numeric(df["parameters_(millions)"], downcast="integer")
df.drop(columns=["size", "parameters"], inplace=True)
df["dependencies"] = df["dependencies"].apply(lambda x: x[1:-1].split(','))
df["analysis"].fillna("unknown", inplace=True)
df["access"] = df["access"].map({"open":2, "limited":1, "closed": 0})
df["training_time"].fillna("unknown", inplace=True)

In [168]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 359 entries, 3 to 564
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   type                   359 non-null    object        
 1   name                   359 non-null    object        
 2   organization           359 non-null    object        
 3   description            299 non-null    object        
 4   created_date           357 non-null    datetime64[ns]
 5   url                    357 non-null    object        
 6   analysis               359 non-null    object        
 7   dependencies           359 non-null    object        
 8   quality_control        85 non-null     object        
 9   access                 359 non-null    int64         
 10  license                343 non-null    object        
 11  intended_uses          129 non-null    object        
 12  prohibited_uses        86 non-null     object        
 13  monitoring

In [169]:
df["training_hardware"]

3                         A single NVIDIA Tesla-P100 GPU
4                                                    NaN
6                                                unknown
7                                      2x A100 80GB GPUs
11                                                   NaN
                             ...                        
559                                              unknown
561                                              unknown
562                                   8 x A100 40GB GPUs
563    56 DGX A100 nodes, each equipped with 4 80GB A...
564    Trained on the Cerebras Condor Galaxy 1 (CG-1)...
Name: training_hardware, Length: 359, dtype: object

In [119]:
df.sort_values("parameters_(millions)", ascending=False)[["name", "organization", "parameters_(millions)"]].head(50)

,name,organization,parameters_(millions)
524,Megatron-LM,NVIDIA,1000000.0
559,Samba 1,Samba Nova Systems,1000000.0
202,PaLM-E,Google,562000.0
163,Med-PaLM Multimodal,Google,562000.0
169,PaLM-SayCan,Google,540000.0
198,Minerva,Google,540000.0
168,U-PaLM,Google,540000.0
166,Flan-U-PaLM,Google,540000.0
165,Flan-PaLM,Google,540000.0
162,Med-PaLM,Google,540000.0
